In [1]:
# !nvidia-smi

In [2]:
import os
os.getcwd()

'/Users/lhe339/Documents/GitHub/study/Pytorch/week1-DNN'

In [3]:
#라이브러리 불러오기
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data_utils 
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import Dataset, DataLoader, TensorDataset

from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
print (f"PyTorch version:{torch.__version__}") # 1.12.1 이상
print(f"MPS 장치를 지원하도록 build 되었는지: {torch.backends.mps.is_built()}") # True 여야 합니다.
print(f"MPS 장치가 사용 가능한지: {torch.backends.mps.is_available()}") # True 여야 합니다.
!python -c 'import platform;print(platform.platform())'

PyTorch version:1.13.1
MPS 장치를 지원하도록 build 되었는지: True
MPS 장치가 사용 가능한지: True
macOS-13.0-arm64-arm-64bit


In [4]:
#데이터 불러오기
data_cov = pd.read_csv('covtype.csv')
data_cov['Cover_Type'].value_counts()
#class1 == 3, class0 == 4
data1= data_cov[data_cov['Cover_Type'] == 3]
data0 = data_cov[data_cov['Cover_Type'] == 4]

df = pd.concat([data1,data0])

dataset=df.sample(frac=1).reset_index(drop=True) #샘플 섞어주기
dataset.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,2623,194,12,404,3,1664,218,250,164,1879,...,0,0,0,0,0,0,0,0,0,3
1,2526,228,18,67,12,1075,190,254,196,1639,...,0,0,0,0,0,0,0,0,0,3
2,2650,82,8,424,43,1031,232,225,126,934,...,0,0,0,0,0,0,0,0,0,3
3,2126,10,16,0,0,834,202,207,140,582,...,0,0,0,0,0,0,0,0,0,3
4,2368,57,11,180,8,376,228,216,120,700,...,0,0,0,0,0,0,0,0,0,3


In [5]:
dataset['Cover_Type'].replace([3, 4],[1, 0],inplace=True)

In [6]:
dataset['Cover_Type'].value_counts()

1    35754
0     2747
Name: Cover_Type, dtype: int64

In [7]:
X = dataset.drop('Cover_Type', axis =1)
y = dataset['Cover_Type']

In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)

X_scaled = scaler.transform(X) 

scaled_df = pd.DataFrame(X_scaled, columns=X.columns) 
scaled_df.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40
0,1.224633,0.191400,-0.951564,1.391801,-0.978979,1.202810,0.352511,1.241121,0.486867,1.854108,...,0.0,-0.052543,-0.011397,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.731035,0.509983,-0.288476,-0.940593,-0.826885,0.221817,-0.342776,1.386750,1.093080,1.396182,...,0.0,-0.052543,-0.011397,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.362026,-0.858049,-1.393623,1.530222,-0.303006,0.148534,0.700154,0.330940,-0.233011,0.051025,...,0.0,-0.052543,-0.011397,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-1.304421,-1.532695,-0.509506,-1.404303,-1.029677,-0.179575,-0.044796,-0.324391,0.032207,-0.620600,...,0.0,-0.052543,-0.011397,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.072971,-1.092301,-1.062079,-0.158514,-0.894482,-0.942385,0.600828,0.003275,-0.346676,-0.395453,...,0.0,-0.052543,-0.011397,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
dataset = pd.concat([scaled_df,y],1)

/var/folders/0p/2db9ylzj0cd11s7tz69zggtm0000gn/T/ipykernel_3238/1137587587.py:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  dataset = pd.concat([scaled_df,y],1)


In [10]:
# 신규 (테스트) 데이터 생성 
data = dataset.sample(frac=0.6, random_state=786)
data_test = dataset.drop(data.index)

data.reset_index(drop=True, inplace=True)
data_test.reset_index(drop=True, inplace=True)

print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_test.shape))

Data for Modeling: (23101, 55)
Unseen Data For Predictions: (15400, 55)


In [11]:
train, val = train_test_split(data, test_size = 0.3, random_state = 9492)

In [12]:
device = torch.device('mps:0' if torch.backends.mps.is_available() else 'cpu') #gpu를 사용할 수 있으면 사용하고, 아니면 cpu를 사용하겠다
print(f"device: {device}")
train_x = train.iloc[:,:-1]
train_y = train.iloc[:,-1]
val_x = val.iloc[:,:-1]
val_y = val.iloc[:,-1]

trn_y  = torch.FloatTensor(train_y.values)
trn_X  = torch.FloatTensor(train_x.values)
val_y  = torch.FloatTensor(val_y.values)
val_X  = torch.FloatTensor(val_x.values)


train = TensorDataset(trn_X, trn_y)
val  = TensorDataset(val_X, val_y)


batch_size = 64
train_kwargs = {'dataset': train,
                'batch_size': batch_size,
               'shuffle': True}
test_kwargs = {'dataset': val,
                'batch_size': batch_size,
              'shuffle': False}


train_loader = torch.utils.data.DataLoader(**train_kwargs)
val_loader  = torch.utils.data.DataLoader(**test_kwargs)

device: mps:0


In [13]:
# gpu 사용
use_mps = torch.backends.mps.is_available()

In [14]:
# class DNN(nn.Module):
    
#     def __init__(self, input_size, output_size, drop_rate):
#         super(DNN, self).__init__() #nn.Module 상속 받음
        
#         layer1 = [nn.Linear(input_size, 48),
#                  nn.BatchNorm1d(48),
#                  nn.Dropout(drop_rate),
#                  nn.ReLU()]
        
#         layer2 = [nn.Linear(48, 24),
#                  nn.BatchNorm1d(24),
#                  nn.Dropout(drop_rate),
#                  nn.ReLU()]
        
#         layer3 = [nn.Linear(24, 12),
#                  nn.BatchNorm1d(12),
#                  nn.Dropout(drop_rate),
#                  nn.ReLU()]
        
#         layer4 = [nn.Linear(12, output_size),
#                  nn.Sigmoid()]
        
#         self.fisrt_layer = nn.Sequential(*layer1)
#         self.second_layer = nn.Sequential(*layer2)
#         self.third_layer = nn.Sequential(*layer3)
#         self.last_layer = nn.Sequential(*layer4)
        
#     def forward(self, x):
#         x = self.fisrt_layer(x)
#         x = self.second_layer(x)
#         x = self.third_layer(x)
#         out = self.last_layer(x)

#         return out 

In [15]:
# model_kwargs = {'drop_rate': 0.56,
#               'input_size': 54,
#                'output_size': 1}

# training_kwargs = {'learning_rate': 1e-3,
#               'epoch': 100}

# model = DNN(**model_kwargs).to(device)

# criterion = nn.BCELoss()

# optimizer = optim.Adam(model.parameters(), lr=training_kwargs['learning_rate'])

# num_epochs = training_kwargs['epoch']

- nn.BCELoss()는 이진 분류(binary classification) 문제에서 사용하는 손실 함수 중 하나입니다.<br> BCE는 Binary Cross Entropy의 약자이며, 이 함수는 실제 타겟값과 예측값 사이의 차이를 계산합니다.<br>

- 이 손실 함수는 주로 출력값이 0 또는 1인 이진 분류 문제에서 사용됩니다.<br> 예를 들어, 스팸 메일 분류 문제에서 이메일이 스팸인지 아닌지를 예측하는 이진 분류 문제를 생각해 볼 수 있습니다.<br> 이 경우, nn.BCELoss() 함수는 모델의 출력값과 실제 타겟값 간의 차이를 측정하여 모델을 훈련합니다.<br>

- 이 손실 함수를 사용하는 이유는 모델이 예측을 잘못할 때 더 큰 페널티를 부여하기 위해서입니다.<br> BCELoss 함수는 모델이 예측을 맞춘 경우에는 작은 손실값을 반환하고, 예측을 잘못한 경우에는 더 큰 손실값을 반환합니다.<br> 따라서 모델은 정확한 예측을 하기 위해 노력하게 됩니다.<br>

- 이진 분류 문제에서 nn.BCELoss() 함수는 일반적으로 시그모이드 활성화 함수와 함께 사용됩니다.<br> 시그모이드 함수는 0과 1 사이의 값을 출력하기 때문에, 이진 분류 문제에서 모델의 출력값을 나타내는데 적합합니다.<br> nn.BCELoss() 함수는 모델의 출력값과 타겟값 간의 차이를 계산하여 손실값을 반환합니다.<br> 이 손실값은 모델을 훈련하는 동안 역전파(backpropagation)를 통해 사용되며, 모델의 가중치를 업데이트합니다.

- nn.BCELoss() 함수의 수식은 다음과 같습니다.

$$(x, y) = -\frac{1}{N}\sum_{i=1}^{N}(y_i\log(x_i) %2B (1-y_i)\log(1-x_i))"$$

- 여기서 $x$는 모델이 예측한 이진 분류 결과이고, $y$는 실제 이진 분류 결과입니다.<br> 이 식은 모든 데이터 포인트 $i$에 대한 손실값을 계산합니다.

- BCELoss 함수는 각 데이터 포인트 $i$의 손실을 계산하기 위해 크로스 엔트로피(cross-entropy) 손실 함수를 사용합니다.<br> 이 손실 함수는 실제 타겟값과 예측값 사이의 차이를 계산하는데, 이진 분류 문제에서는 $y$가 $0$일 때는 $1-x$, $y$가 $1$일 때는 $x$를 계산하여 사용합니다.

- 이 손실 함수는 모델의 예측값이 실제 타겟값과 가까울수록 손실값이 작아지도록 학습됩니다.<br> 모델이 정확하게 예측하는 경우 손실값은 $0$에 가까워집니다.

- Adam(Adaptive Moment Estimation)은 최적화 알고리즘 중 하나로, 딥러닝 모델 학습에 널리 사용됩니다.<br> Adam은 RMSprop과 모멘텀 최적화 방법을 결합한 알고리즘으로, 각각의 파라미터마다 적응적인 학습률(adaptive learning rate)을 사용하여 학습을 진행합니다.

- Adam의 핵심 아이디어는 다음과 같습니다.

1. 모멘텀 최적화 방법처럼, 이전 스텝에서의 gradient 정보를 이용하여 이번 스텝에서의 gradient 값을 업데이트합니다.<br> 이를 통해 일정 방향으로 gradient를 누적하면서 빠르게 수렴하도록 합니다.

2. RMSprop 방법처럼, 각각의 파라미터마다 적응적인 학습률을 적용합니다.<br> 즉, 각 파라미터마다 gradient 크기에 대한 이동 평균을 구하고, 이를 통해 학습률을 조절합니다.<br> 이를 통해 각 파라미터마다 적절한 학습률을 적용하여, 수렴 속도와 성능을 개선할 수 있습니다.

3. 위의 방법들을 조합하여, 이전 gradient의 이동 평균과 누적된 gradient 값을 이용하여 적응적인 학습률을 계산합니다.<br> 즉, 이전 스텝에서의 gradient 정보와 현재 스텝에서의 gradient 정보를 이용하여, 보다 빠르고 안정적으로 수렴하도록 합니다.

- Adam은 이러한 아이디어를 바탕으로, 적응적인 학습률을 사용하면서도 모멘텀 최적화 방법과 비슷한 수렴 속도와 성능을 보이는 것으로 알려져 있습니다.<br> 따라서, 딥러닝 모델 학습에 자주 사용되는 최적화 알고리즘 중 하나입니다.

In [16]:
# print('Neural network')
# print(model)

In [17]:
# import copy
# import warnings
# warnings.filterwarnings("ignore") 
# #loss
# criterion = nn.BCELoss()

# def train(model, device, train_loader, optimizer, epoch):
#     global criterion
#     model.train()
#     for batch_idx, (data, target) in enumerate(train_loader):
#         data, target = data.to(device), target.to(device)
#         optimizer.zero_grad()
#         output = model(data) #output: 64 x 1
#         output = output.squeeze(1) #output: 64
#         loss = criterion(output, target)
#         loss.backward()
#         optimizer.step()
#         if batch_idx % 100 == 0:
#             print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
#                 epoch, batch_idx * len(data), len(train_loader.dataset),
#                 100. * batch_idx / len(train_loader), loss.item()))


# def test():
#     global criterion
#     model.eval()
#     with torch.no_grad():
#         n_correct = 0
#         n_samples = 0
#         for batch_idx, (data, target) in enumerate (val_loader):
#             valid_loss = {'loss_val' : []}
            
#             data, target = data.to(device), target.to(device)
#             outputs = model(data)
#             outputs = outputs.squeeze()
            
#             loss_val = criterion(outputs, target)
#             valid_loss['loss_val'].append(loss_val.item())
            
#             outputs = torch.flatten(outputs).cpu().detach().numpy()
#             target = torch.flatten(target).cpu().detach().numpy()
#             predicted = np.where(outputs>0.5, 1, 0)
            
#             n_samples += len(target)
#             n_correct += (predicted == target).sum().item()
#             vl_mean = np.array(valid_loss['loss_val']).mean()
            
#             if batch_idx % 100 == 0:
#                 print('Batch mean Validation Loss: {:.6f}'.format(vl_mean))

#     acc = 100.0 * n_correct / n_samples
#     print(f'Accuracy of the network on the 10000 validation set: {acc} %')
#     return acc

- 위 코드에서 copy 모듈은 딥러닝 모델의 가중치(weight)와 편향(bias) 등의 매개변수를 복사(copy)하는 데 사용됩니다.

- best_model = copy.deepcopy(model)은 현재의 딥러닝 모델 model을 깊은 복사(deep copy)하여 best_model에 저장합니다.<br> 깊은 복사는 객체를 완전히 새로운 객체로 복사하는 방법으로, 원본 객체와 복사본 객체가 서로 독립적이고 다른 객체가 됩니다.<br> 이렇게 복사본을 생성하는 이유는, 만약 모델의 매개변수를 직접 수정하면 원본 모델도 영향을 받기 때문입니다.<br> 이러한 문제를 피하기 위해, 깊은 복사를 통해 복사본을 생성하고 복사본을 수정하여 사용합니다.

- 따라서, 위 코드에서는 현재 모델의 성능이 이전 모델의 성능보다 우수할 경우, 현재 모델의 매개변수를 best_model에 저장합니다.<br> 이후 best_model은 이전의 모델과는 완전히 별개의 객체이기 때문에, 이후에 이 객체를 사용하여 학습을 진행해도 이전의 모델에는 영향을 미치지 않습니다.

- 딥러닝 모델을 학습할 때, 손실 함수(loss function)는 학습하는 동안 반복적으로 사용되는 함수 중 하나입니다.<br> 따라서, 일반적으로 손실 함수는 딥러닝 모델과 별도로 정의하여 사용됩니다.<br> 위 코드에서 criterion = nn.BCELoss()는 이러한 손실 함수를 정의한 부분입니다.

- train() 함수에서 global criterion이라는 문장이 포함된 이유는, train() 함수 내에서 손실 함수를 사용하기 위해 전역 변수(global variable)로서 criterion 변수를 사용하려는 것입니다.<br> 이렇게 전역 변수로 선언해두면, train() 함수 내에서 따로 criterion을 지정해주지 않아도 전역 변수로 정의된 손실 함수를 사용할 수 있습니다.

- 딥러닝 모델에서 사용되는 매개변수와 손실 함수는 딥러닝 모델의 학습에 있어 중요한 역할을 합니다.<br> 따라서, 이러한 매개변수와 손실 함수는 모델 학습 시 정의되어 있어야 합니다.

In [18]:
# OUTPUT_PATH = '/Users/lhe339/Documents/GitHub/study/Pytorch/week1-DNN'
# best_acc = -1
# best_model = None
# for epoch in range(0, num_epochs):
#     train(model, device, train_loader, optimizer, epoch)
#     acc = test()
#     if acc > best_acc:
#         best_acc = acc
#         best_model = copy.deepcopy(model)
#         print(f'new best acc={best_acc}')
#     else:
#         print(f'current acc={acc}, prev_best_acc={best_acc}')

# state = {
#     'model_state': best_model.state_dict(),
#     'test_acc': best_acc
# }
# torch.save(state, OUTPUT_PATH+'/practice_nn.pt')  #훈련 끝나고 모델을 저장
# print('best model saved')

In [19]:
# #new data inference

# x = torch.normal(mean=6.9671e-06, std=0.4746, size=(1,54))
# out = model(x.to(device))
# out = torch.flatten(out).cpu().detach().numpy()
# predicted = np.where(out>0.5, 1, 0)

In [20]:
# print('예측 클래스:', predicted)

## [HW] 3-layer neural network 를 구현하고 학습하세요(batch norm, drop out, activation function 자유 구성)

In [21]:
class threee_layer_nn(nn.Module):
    
    def __init__(self, input_size, output_size, drop_rate):
        super(threee_layer_nn, self).__init__()
        
        layer1 = [nn.Linear(input_size, 128),
                 nn.BatchNorm1d(128),
                 nn.Dropout(drop_rate),
                 nn.ReLU()]

        layer2 = [nn.Linear(128, 64),
                 nn.BatchNorm1d(64),
                 nn.Dropout(drop_rate),
                 nn.ReLU()]
        
        layer3 = [nn.Linear(64, output_size),
                 nn.Sigmoid()]
        
        self.first_layer = nn.Sequential(*layer1)
        self.second_layer = nn.Sequential(*layer2)
        self.last_layer = nn.Sequential(*layer3)
        
    def forward(self, x):
        x = self.first_layer(x)
        x = self.second_layer(x)
        out = self.last_layer(x)

        return out

In [22]:
model_kwargs = {'drop_rate' : 0.2,
            'input_size' : 256,
            'output_size' : 2}

training_kwargs = {'learning_rate' : 1e-2,
                 'epoch' : 10}

model = threee_layer_nn(**model_kwargs).to(device)

criterion = nn.BCELoss()

optimizer = optim.Adam(model.parameters(), lr=training_kwargs['learning_rate'])

num_epochs = training_kwargs['epoch']

In [23]:
print(model)

threee_layer_nn(
  (first_layer): Sequential(
    (0): Linear(in_features=256, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Dropout(p=0.2, inplace=False)
    (3): ReLU()
  )
  (second_layer): Sequential(
    (0): Linear(in_features=128, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Dropout(p=0.2, inplace=False)
    (3): ReLU()
  )
  (last_layer): Sequential(
    (0): Linear(in_features=64, out_features=2, bias=True)
    (1): Sigmoid()
  )
)


In [24]:
import copy
import warnings
warnings.filterwarnings("ignore")

#loss
criterion = nn.BCELoss()

def train(model, device, train_loader, optimizer, epoch):
    global criterion
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        output = output.squeeze(1)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {}[{}/{} ({:.0f}%)]\tLoss: {.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item()))
            
def test():
    global criterion
    model.eval()
    with torch.no_grad():
        n_correct = 0
        n_samples = 0
        for batch_idx, (data, target) in enumerate (val_loader):
            valid_loss = {'loss_val' : []}
            
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            outputs = outputs.squeeze()
            
            loss_val = criterion(outputs, target)
            valid_loss['loss_val'].append(loss_val.item())
            
            outputs = torch.flatten(outputs).cpu().detach().numpy()
            target = torch.flatten(target).cpu().detach().numpy()
            predicted = np.where(outputs>0.5, 1, 0)
            
            n_samples += len(target)
            n_correct += (predicted == target).sum().item()
            vl_mean = np.array(valid_loss['loss_val']).mean()
            
            if batch_idx % 100 == 0:
                print('Batch mean Validation Loss: {:.6f}'.format(vl_mean))

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the 10000 validation set: {acc} %')
    return acc

In [ ]:
OUTPUT_PATH = '/Users/lhe339/Documents/GitHub/study/Pytorch/week1-DNN'
best_acc = -1
best_model = None
for epoch in range(0, num_epochs):
    train(model, device, train_loader, optimizer, epoch)
    acc = test()
    if acc > best_acc:
        best_acc = acc
        best_model = copy.deepcopy(model)
        print(f'new best acc={best_acc}')
    else:
        print(f'current acc={acc}, prev_best_acc={best_acc}')

state = {
    'model_state': best_model.state_dict(),
    'test_acc': best_acc
}
torch.save(state, OUTPUT_PATH+'/homework_nn.pt')  #훈련 끝나고 모델을 저장
print('best model saved')

In [ ]:
#new data inference

x = torch.normal(mean=6.9671e-06, std=0.4746, size=(1,54))
out = model(x.to(device))
out = torch.flatten(out).cpu().detach().numpy()
predicted = np.where(out>0.5, 1, 0)

In [ ]:
print('예측 클래스:', predicted)